# Part I: Create a data frame for neighborhoods in Philadelphia

In [2]:
import pandas as pd

In [4]:
#read csv
philly_df = pd.read_csv('Philadelphia neighborhoods.csv')
philly_df.head()

,Borough,Neighborhood
0,Center City,Avenue of the Arts
1,Center City,Callowhill
2,Center City,Chinatown
3,Center City,Elfreth's Alley
4,Center City,Fitler Square


In [9]:
#Exract neighborhoods in West Philadelphia
westphilly_df = philly_df.loc[philly_df["Borough"]=="West Philadelphia"]
westphilly_df.reset_index(inplace=True) #reset index
westphilly_df.drop(["index"], axis=1, inplace=True) #drop old index
print(westphilly_df.shape)
westphilly_df.head()

(25, 2)


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Borough,Neighborhood
0,West Philadelphia,Belmont District
1,West Philadelphia,Black Bottom
2,West Philadelphia,Carroll Park
3,West Philadelphia,Cathedral Park
4,West Philadelphia,Cedar Park


# Part II: Get coordinates of each neighborhood in West Philadelphia

In [11]:
#Install Geocoder
import sys
!{sys.executable} -m pip install geocoder

     |████████████████████████████████| 102kB 5.0MB/s ta 0:00:011
     |████████████████████████████████| 92kB 5.6MB/s eta 0:00:011
     |████████████████████████████████| 829kB 2.9MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future


In [22]:
#Get neighborhood coordinates

#This obviously doesn't work at all due to unreliable package!!!
"""
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None
latitudes = []
longitudes = []

# loop until you get the coordinates
for neighborhood in westphilly_df.Neighborhood:
    while(lat_lng_coords is None):
        g = geocoder.google('{} Philadelphia, PA'.format(neighborhood))
        lat_lng_coords = g.latlng
    
    latitude = lat_lng_coords[0]
    latitudes.append(latitude)
    longitude = lat_lng_coords[1]
    longitudes.append(longitude)
"""
#This is an alternative way to get coordinates through wikipedia
coord_df = pd.read_csv('West philly neighborhoods coordinates.csv')
westphilly_df["Latitude"] = coord_df.Latitude
westphilly_df["Longitude"] = coord_df.Longitude

westphilly_df.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Borough,Neighborhood,Latitude,Longitude
0,West Philadelphia,Belmont District,39.966667,-75.2050
1,West Philadelphia,Black Bottom,39.957400,-75.1978
2,West Philadelphia,Carroll Park,39.973000,-75.2360
3,West Philadelphia,Cathedral Park,39.973000,-75.2360
4,West Philadelphia,Cedar Park,39.947000,-75.2160


# Part III: Explore and cluster the neighborhoods in West Philadelphia

In [23]:
# Import all the necessary packages

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0



geopy-1.22.0         | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: 

## 1. Explore neighborhoods in West Philadelphia

In [24]:
#Define Foursquare credentials
CLIENT_ID = 'HJLSQDNYRULF5SAYPOWOPTTPBNXMP4QDNE1FMTUNPYJPGGUP' # your Foursquare ID
CLIENT_SECRET = 'QKY03Y4CXUL4ESLRWFANEGL4FNPKI5GZZDNFU0OHEZSRW5ZD' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HJLSQDNYRULF5SAYPOWOPTTPBNXMP4QDNE1FMTUNPYJPGGUP
CLIENT_SECRET:QKY03Y4CXUL4ESLRWFANEGL4FNPKI5GZZDNFU0OHEZSRW5ZD


In [26]:
#Get coordinates for Downtown Toronto
address = 'West Philadelphia, PA'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of West Philadelphia are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of West Philadelphia are 39.9568763, -75.1846425.


In [30]:
# create map of West Philadelphia using latitude and longitude values
map_westphilly = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(westphilly_df['Latitude'], westphilly_df['Longitude'], westphilly_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_westphilly)  
    
map_westphilly

## 2. Get top 10 venues of every neighborhood in West Philadelphia from Foursquare

In [31]:
#Define a function to get neighborhood venues from Foursquare
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [32]:
#Get venues of all neighborhoods in West Philadelphia from Foursquare
LIMIT = 100
radius = 500
westphilly_venues = getNearbyVenues(names=westphilly_df['Neighborhood'],
                                   latitudes=westphilly_df['Latitude'],
                                   longitudes=westphilly_df['Longitude']
                                  )

Belmont District
Black Bottom
Carroll Park
Cathedral Park
Cedar Park
Cobbs Creek
Dunlap
Garden Court
Haddington
Haverford North
Mantua
Mill Creek
Overbrook
Overbrook Park
Overbrook Farms
Parkside
Powelton Village
Saunders Park
Spruce Hill
Squirrel Hill
University City
Walnut Hill
Woodland Terrace
Wynnefield
Wynnefield Heights


In [33]:
print(westphilly_venues.shape)
westphilly_venues.head()

(562, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Belmont District,39.966667,-75.205,Quality Taste Jamaican Restaurant,39.963130,-75.202695,Caribbean Restaurant
1,Belmont District,39.966667,-75.205,Texas Wieners,39.966669,-75.207636,Breakfast Spot
2,Belmont District,39.966667,-75.205,Cricket Wireless Authorized Retailer,39.963993,-75.204205,Mobile Phone Shop
3,Belmont District,39.966667,-75.205,Bottom of the Sea,39.965419,-75.206941,Seafood Restaurant
4,Belmont District,39.966667,-75.205,LAVA,39.965449,-75.206393,Event Space


In [34]:
print('There are {} uniques categories.'.format(len(westphilly_venues['Venue Category'].unique())))
westphilly_venues.groupby('Neighborhood').count()

There are 135 uniques categories.


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Belmont District,8,8,8,8,8,8
Black Bottom,56,56,56,56,56,56
Carroll Park,12,12,12,12,12,12
Cathedral Park,12,12,12,12,12,12
Cedar Park,34,34,34,34,34,34
Cobbs Creek,5,5,5,5,5,5
Dunlap,23,23,23,23,23,23
Garden Court,26,26,26,26,26,26
Haddington,8,8,8,8,8,8


In [35]:
#Analyze each neighborhood
# one hot encoding
westphilly_onehot = pd.get_dummies(westphilly_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
westphilly_onehot['Neighborhood'] = westphilly_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [westphilly_onehot.columns[-1]] + list(westphilly_onehot.columns[:-1])
westphilly_onehot = westphilly_onehot[fixed_columns]

print(westphilly_onehot.shape)
westphilly_onehot.head()

(562, 136)


,Neighborhood,African Restaurant,American Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Garage,Bagel Shop,Bakery,Bar,Baseball Field,Baseball Stadium,Beer Garden,Boarding House,Bookstore,Breakfast Spot,Bubble Tea Shop,Building,Burger Joint,Bus Station,Bus Stop,Business Service,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Bookstore,College Cafeteria,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Donut Shop,Ethiopian Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Garden,Gas Station,Gastropub,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Hardware Store,Hookah Bar,Hotel,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Lawyer,Light Rail Station,Liquor Store,Lounge,Market,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Moving Target,Music Venue,New American Restaurant,Office,Optical Shop,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pizza Place,Playground,Plaza,Public Art,Ramen Restaurant,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rugby Pitch,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Plaza,Snack Place,Southern / Soul Food Restaurant,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Strip Club,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Shop,Women's Store,Yoga Studio
0,Belmont District,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Belmont District,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Belmont District,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Belmont District,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Belmont District,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [36]:
#Group West Phialdelphia one_hot by Neighborhood
westphilly_grouped = westphilly_onehot.groupby('Neighborhood').mean().reset_index()
westphilly_grouped

,Neighborhood,African Restaurant,American Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Garage,Bagel Shop,Bakery,Bar,Baseball Field,Baseball Stadium,Beer Garden,Boarding House,Bookstore,Breakfast Spot,Bubble Tea Shop,Building,Burger Joint,Bus Station,Bus Stop,Business Service,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Bookstore,College Cafeteria,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Donut Shop,Ethiopian Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Garden,Gas Station,Gastropub,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Hardware Store,Hookah Bar,Hotel,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Lawyer,Light Rail Station,Liquor Store,Lounge,Market,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Moving Target,Music Venue,New American Restaurant,Office,Optical Shop,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pizza Place,Playground,Plaza,Public Art,Ramen Restaurant,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rugby Pitch,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Plaza,Snack Place,Southern / Soul Food Restaurant,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Strip Club,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Shop,Women's Store,Yoga Studio
0,Belmont District,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.125000,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
1,Black Bottom,0.000000,0.000000,0.000000,0.017857,0.000000,0.017857,0.0,0.00,0.000000,0.017857,0.0,0.000000,0.017857,0.017857,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.017857,0.000000,0.017857,0.000000,0.053571,0.000000,0.00,0.017857,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.053571,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.053571,0.000000,0.017857,0.000000,0.000000,0.017857,0.000000,0.000000,0.000000,0.035714,0.000000,0.00,0.000000,0.017857,0.017857,0.017857,0.017857,0.035714,0.000000,0.000000,0.017857,0.000000,0.035714,0.000000,0.000000,0.000000,0.017857,0.000000,0.000000,0.000000,0.000000,0.017857,0.000000,0.00,0.000000,0.017857,0.000000,0.000000,0.017857,0.000000,0.035714,0.000000,0.017857,0.000000,0.000000,0.071429,0.000000,0.

In [37]:
#Display top 10 venues for each neighborhood
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = westphilly_grouped['Neighborhood']

for ind in np.arange(westphilly_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(westphilly_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Belmont District,Breakfast Spot,Seafood Restaurant,Pizza Place,Mobile Phone Shop,Light Rail Station,Caribbean Restaurant,Event Space,Yoga Studio,Flea Market,Fast Food Restaurant
1,Black Bottom,Pizza Place,Coffee Shop,Donut Shop,Food Truck,Indian Restaurant,Sandwich Place,Gym,Performing Arts Venue,Korean Restaurant,Restaurant
2,Carroll Park,Southern / Soul Food Restaurant,Light Rail Station,Deli / Bodega,Park,Intersection,Pizza Place,Asian Restaurant,Breakfast Spot,Chinese Restaurant,Food
3,Cathedral Park,Southern / Soul Food Restaurant,Light Rail Station,Deli / Bodega,Park,Intersection,Pizza Place,Asian Restaurant,Breakfast Spot,Chinese Restaurant,Food
4,Cedar Park,Ethiopian Restaurant,Indian Restaurant,Chinese Restaurant,Grocery Store,Dive Bar,Convenience Store,Deli / Bodega,Playground,Pizza Place,Dessert Shop


## 3. Cluster neighborhoods in West Philadelphia

In [38]:
# set number of clusters
kclusters = 5

westphilly_grouped_clustering = westphilly_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(westphilly_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 1, 2, 2, 1, 1, 1, 1, 2, 1], dtype=int32)

In [39]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

westphilly_merged = westphilly_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
westphilly_merged = westphilly_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

westphilly_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,West Philadelphia,Belmont District,39.966667,-75.2050,2,Breakfast Spot,Seafood Restaurant,Pizza Place,Mobile Phone Shop,Light Rail Station,Caribbean Restaurant,Event Space,Yoga Studio,Flea Market,Fast Food Restaurant
1,West Philadelphia,Black Bottom,39.957400,-75.1978,1,Pizza Place,Coffee Shop,Donut Shop,Food Truck,Indian Restaurant,Sandwich Place,Gym,Performing Arts Venue,Korean Restaurant,Restaurant
2,West Philadelphia,Carroll Park,39.973000,-75.2360,2,Southern / Soul Food Restaurant,Light Rail Station,Deli / Bodega,Park,Intersection,Pizza Place,Asian Restaurant,Breakfast Spot,Chinese Restaurant,Food
3,West Philadelphia,Cathedral Park,39.973000,-75.2360,2,Southern / Soul Food Restaurant,Light Rail Station,Deli / Bodega,Park,Intersection,Pizza Place,Asian Restaurant,Breakfast Spot,Chinese Restaurant,Food
4,West Philadelphia,Cedar Park,39.947000,-75.2160,1,Ethiopian Restaurant,Indian Restaurant,Chinese Restaurant,Grocery Store,Dive Bar,Convenience Store,Deli / Bodega,Playground,Pizza Place,Dessert Shop


In [41]:
# Visualize the resuting clusters
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(westphilly_merged['Latitude'], westphilly_merged['Longitude'], westphilly_merged['Neighborhood'], westphilly_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 4. Examine Clusters

In [42]:
#Cluster 1: Pizza place
westphilly_merged.loc[westphilly_merged['Cluster Labels'] == 0, westphilly_merged.columns[[2] + list(range(5, westphilly_merged.shape[1]))]]

,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,39.964000,Pizza Place,Bakery,Light Rail Station,Photography Studio,Chinese Restaurant,Gym,Event Space,Food,Flea Market,Fast Food Restaurant
22,39.948889,Pizza Place,Coffee Shop,Bar,Arcade,Park,Bookstore,Mexican Restaurant,College Bookstore,Farmers Market,Sandwich Place
23,39.989000,Pizza Place,Food,Moving Target,Salon / Barbershop,Yoga Studio,Ethiopian Restaurant,Flea Market,Fast Food Restaurant,Farmers Market,Event Space


In [43]:
#Cluster 2: Core of the borough
westphilly_merged.loc[westphilly_merged['Cluster Labels'] == 1, westphilly_merged.columns[[2] + list(range(5, westphilly_merged.shape[1]))]]

,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,39.957400,Pizza Place,Coffee Shop,Donut Shop,Food Truck,Indian Restaurant,Sandwich Place,Gym,Performing Arts Venue,Korean Restaurant,Restaurant
4,39.947000,Ethiopian Restaurant,Indian Restaurant,Chinese Restaurant,Grocery Store,Dive Bar,Convenience Store,Deli / Bodega,Playground,Pizza Place,Dessert Shop
5,39.952912,Sandwich Place,Seafood Restaurant,Park,Spanish Restaurant,Intersection,Dessert Shop,Diner,Discount Store,Dive Bar,Donut Shop
6,39.961000,Bar,Clothing Store,Breakfast Spot,Pizza Place,Shoe Store,Discount Store,Caribbean Restaurant,Chinese Restaurant,Food,Lounge
7,39.951944,Deli / Bodega,Indian Restaurant,Chinese Restaurant,Middle Eastern Restaurant,Sandwich Place,Mexican Restaurant,Diner,Flea Market,Seafood Restaurant,Caribbean Restaurant
9,39.965300,Breakfast Spot,Recreation Center,Pizza Place,Intersection,Art Gallery,Mobile Phone Shop,Tram Station,Discount Store,Caribbean Restaurant,Bakery
13,39.977000,Pharmacy,Fried Chicken Joint,American Restaurant,Video Store,Diner,Pizza Place,Fast Food Restaurant,Bagel Shop,Grocery Store,Event Space
16,39.959722,Food Truck,Pizza Place,Sandwich Place,Coffee Shop,Recreation Center,Bubble Tea Shop,Burger Joint,Convenience Store,Creperie,Dessert Shop
17,39.959000,Donut Shop,Cosmetics Shop,Coffee Shop,Lounge,Tapas Restaurant,Performing Arts Venue,Office,Chinese Restaurant,Sandwich Place,Café
18,39.954000,Middle Eastern Restaurant,Pizza Place,Hookah Bar,Gas Station,Art Gallery,Vietnamese Restaurant,Pakistani Restaurant,Grocery Store,Ethiopian Restaurant,Café


In [44]:
#Cluster 3: Transportation center
westphilly_merged.loc[westphilly_merged['Cluster Labels'] == 2, westphilly_merged.columns[[2] + list(range(5, westphilly_merged.shape[1]))]]

,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,39.966667,Breakfast Spot,Seafood Restaurant,Pizza Place,Mobile Phone Shop,Light Rail Station,Caribbean Restaurant,Event Space,Yoga Studio,Flea Market,Fast Food Restaurant
2,39.973000,Southern / Soul Food Restaurant,Light Rail Station,Deli / Bodega,Park,Intersection,Pizza Place,Asian Restaurant,Breakfast Spot,Chinese Restaurant,Food
3,39.973000,Southern / Soul Food Restaurant,Light Rail Station,Deli / Bodega,Park,Intersection,Pizza Place,Asian Restaurant,Breakfast Spot,Chinese Restaurant,Food
8,39.961389,Intersection,Bakery,Dance Studio,Dessert Shop,Pizza Place,Seafood Restaurant,Sports Bar,Deli / Bodega,Gym,Hotpot Restaurant
12,39.988000,Deli / Bodega,Intersection,Pizza Place,Breakfast Spot,Southern / Soul Food Restaurant,Indian Restaurant,Train Station,Train,Trail,Gym
14,39.986389,Intersection,Pizza Place,Deli / Bodega,Chinese Restaurant,Farmers Market,Southern / Soul Food Restaurant,Trail,Dessert Shop,Diner,Design Studio


In [45]:
#Cluster 4: art and garage
westphilly_merged.loc[westphilly_merged['Cluster Labels'] == 3, westphilly_merged.columns[[2] + list(range(5, westphilly_merged.shape[1]))]]

,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,39.973889,Bar,Baseball Field,Art Gallery,Auto Garage,Sculpture Garden,Food Truck,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant


In [46]:
#Cluster 5: shopping
westphilly_merged.loc[westphilly_merged['Cluster Labels'] == 4, westphilly_merged.columns[[2] + list(range(5, westphilly_merged.shape[1]))]]

,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,39.966,Intersection,Discount Store,Bus Stop,Convenience Store,Park,Liquor Store,Fast Food Restaurant,Food & Drink Shop,Food,Flea Market


In [50]:
# Save the examine results to a csv

westphilly_outout = westphilly_merged.sort_values(by=["Cluster Labels"])
westphilly_outout.to_csv("clustered neighborhood_west philly.csv", index=False)